In [3]:
import pandas as pd

df = pd.read_csv('data/name_gender_filtered.csv')
unique_chars = set()

for name in df['Name']:
    unique_chars.update(name)
unique_chars = sorted(list(unique_chars))
unique_chars = ''.join(unique_chars)
print(unique_chars)

abcdefghijklmnopqrstuvwxyz


In [4]:
import torch

n_letters = len(unique_chars)
def nameToTensor(name):
    tensor = torch.zeros(len(name), n_letters)
    for char_idx, char in enumerate(name):
        letter_idx = unique_chars.find(char)
        assert letter_idx != -1, f"char is {name}, {char}"
        tensor[char_idx][letter_idx] = 1
    return tensor

gen2num = {'F':0, 'M':1}
num2gen = {0:'F', 1:'M'}

In [7]:
# tensor check

shuffled_df = df.sample(frac=1).reset_index(drop=True)

for index, row in shuffled_df.iterrows():
    input_tensor = nameToTensor(row['Name'])
    target_tensor = torch.tensor([gen2num[row['Gender']]], dtype=torch.long)
    
    print(row['Name'])
    print(input_tensor)
    print(row['Gender'], gen2num[row['Gender']])
    print(target_tensor)
    break

buffie
tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
F 0
tensor([0])


In [8]:
import torch
from torch.optim import Adam, SGD

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(rnn_model.parameters(), lr=0.0001)

rnn_model.train()
for epoch_idx in range(200):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)

    total_loss = 0.
    correct_predictions = 0
    total_predictions = 0

    for index, row in shuffled_df.iterrows():
        input_tensor = nameToTensor(row['Name'])
        target_tensor = torch.tensor([gen2num[row['Gender']]], dtype=torch.long)

        hidden = rnn_model.get_hidden()

        rnn_model.zero_grad()

        for char_idx in range(input_tensor.size()[0]):
            char_tensor = input_tensor[char_idx]
            output, hidden = rnn_model(char_tensor[None,:], hidden)

        loss = loss_fn(output, target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted_index = torch.argmax(output, 1)
        correct_predictions += (predicted_index == target_tensor).sum().item()
        total_predictions += 1

    average_loss = total_loss / total_predictions
    accuracy = 100 * correct_predictions / total_predictions
    print(f'Epoch: {epoch_idx}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')


/Users/hyungseok/anaconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch: 0, Loss: 0.4530, Accuracy: 78.38%
Epoch: 1, Loss: 0.3868, Accuracy: 82.95%
Epoch: 2, Loss: 0.3786, Accuracy: 83.20%
Epoch: 3, Loss: 0.3738, Accuracy: 83.90%
Epoch: 4, Loss: 0.3709, Accuracy: 83.87%
Epoch: 5, Loss: 0.3700, Accuracy: 83.81%
Epoch: 6, Loss: 0.3692, Accuracy: 84.01%
Epoch: 7, Loss: 0.3684, Accuracy: 83.91%
Epoch: 8, Loss: 0.3679, Accuracy: 83.88%
Epoch: 9, Loss: 0.3671, Accuracy: 84.05%
Epoch: 10, Loss: 0.3669, Accuracy: 84.09%
Epoch: 11, Loss: 0.3665, Accuracy: 84.12%
Epoch: 12, Loss: 0.3661, Accuracy: 84.10%
Epoch: 13, Loss: 0.3654, Accuracy: 84.08%
Epoch: 14, Loss: 0.3646, Accuracy: 84.17%
Epoch: 15, Loss: 0.3642, Accuracy: 84.31%
Epoch: 16, Loss: 0.3637, Accuracy: 84.26%
Epoch: 17, Loss: 0.3636, Accuracy: 84.10%
Epoch: 18, Loss: 0.3627, Accuracy: 84.22%
Epoch: 19, Loss: 0.3621, Accuracy: 84.37%
Epoch: 20, Loss: 0.3613, Accuracy: 84.32%
Epoch: 21, Loss: 0.3607, Accuracy: 84.34%
Epoch: 22, Loss: 0.3592, Accuracy: 84.29%
Epoch: 23, Loss: 0.3584, Accuracy: 84.42%
Ep

In [ ]:
test_name = 'elsa'
test_name_tensor = nameToTensor(test_name)

rnn_model.eval()
hiddne = rnn_model.get_hidden()
for char_idx in range(len(test_name)):
    char_tensor = test_name_tensor[char_idx]
    output, hidden = rnn_model(char_tensor[None,:],hidden)
predicted_index = torch.argmax(output, 1).item()
print(num2gen[predicted_index])
